In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "../../scripts/methods/")
from my_slat import slat_align, slat_align_ref
sys.path.insert(0, "/home/ylu/project")
from utils import *
import time


%load_ext autoreload
%autoreload 2

In [2]:
data_folder = "./results/split_slices_CCF/"
results_folder = "./results/SLAT/"
figures_folder = "./results/figures/SLAT"
Path(results_folder).mkdir(parents=True, exist_ok=True)

In [3]:
import re
def list_and_sort_files_by_number(directory_path, suffix):
    def extract_number(filename):
        match = re.search(r'\d+', filename)
        return int(match.group()) if match else float('inf')
    try:
        files = [f for f in os.listdir(directory_path) if f.endswith(suffix)]
        sorted_files = sorted(files, key=extract_number)
        return sorted_files
    except FileNotFoundError:
        return f"The directory {directory_path} does not exist."
    except NotADirectoryError:
        return f"The path {directory_path} is not a directory."
    except PermissionError:
        return f"Permission denied to access {directory_path}."

directory_path = data_folder
sorted_files = list_and_sort_files_by_number(directory_path, '.h5ad')

In [4]:
## load the data
from tqdm import tqdm
slices = []
for f in tqdm(sorted_files):
    slices.append(ad.read_h5ad(os.path.join(data_folder, f)))

100%|███████████████| 129/129 [00:24<00:00,  5.36it/s]


In [5]:
spatial_key = "spatial_2D"
anno_key = "cell_type"

In [6]:
## Rotate the data
rotate_key = "spatial_rot"
rotations = np.load("./results/random_rotations.npy", allow_pickle=True)
for i in range(len(slices)):
    slices[i].obsm[rotate_key] = slices[i].obsm[spatial_key][:,:2].copy()
    mean = np.mean(slices[i].obsm[rotate_key], axis=0)
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] - mean
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] @ rotations[i].T + mean
    

In [7]:
spatial_key = "spatial_rot"
key_added = "aligned_spatial"

In [8]:
## Perform the SLAT alignment
sampling_num = 20000
for i in tqdm(range(len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    time_start = time.time()
    align_slices, pis = slat_align(
        models = [slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
    )
    time_end = time.time()
    matches = pis
    R, t=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                         | 0/128 [00:00<?, ?it/s]

Calculating spatial neighbor graph ...
The graph contains 143935 edges, 12531 cells.
11.486313941425266 neighbors per cell on average.
Calculating spatial neighbor graph ...
The graph contains 159512 edges, 13952 cells.
11.432912844036696 neighbors per cell on average.
Use DPCA feature to format graph


/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


Warning! Dual PCA is using GPU, which may lead to OUT OF GPU MEMORY in big dataset!
Choose GPU:0 as device
Running
---------- epochs: 1 ----------
---------- epochs: 2 ----------
---------- epochs: 3 ----------
---------- epochs: 4 ----------
---------- epochs: 5 ----------
---------- epochs: 6 ----------
Training model time: 3.02


  1%|▏                | 1/128 [00:23<49:34, 23.42s/it]

...

 99%|██████████████▉| 127/128 [48:29<00:18, 18.39s/it]

Calculating spatial neighbor graph ...
The graph contains 193153 edges, 16887 cells.
11.437970036122461 neighbors per cell on average.
Calculating spatial neighbor graph ...
The graph contains 157573 edges, 13675 cells.
11.522705667276051 neighbors per cell on average.
Use DPCA feature to format graph


/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/ylu/anaconda3/envs/scSLAT/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


Warning! Dual PCA is using GPU, which may lead to OUT OF GPU MEMORY in big dataset!
Choose GPU:0 as device
Running
---------- epochs: 1 ----------
---------- epochs: 2 ----------
---------- epochs: 3 ----------
---------- epochs: 4 ----------
---------- epochs: 5 ----------
---------- epochs: 6 ----------
Training model time: 1.28


100%|███████████████| 128/128 [48:43<00:00, 22.84s/it]
